# Merge Dialogs

In [1]:
import sys

from glob import glob
import os.path

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', None)

sys.path.append('./meetup/code')
from slurk_utils import logs_to_dfs, postproc_df, pp, parse_event_mu, reformat_dial_pp, plot_ade_row

/home/labicquette/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
mu_path = './meetup/data/meetupX'

dial_days = [
    '2018-11-29',
    '2018-11-30',
    '2018-12-01',
    '2018-12-02']

dials = [p for s in [glob(os.path.join(mu_path, good_dial, '*log')) for good_dial in dial_days] for p in s]

dial, meta_data = logs_to_dfs(dials, parser=parse_event_mu)
dial = [postproc_df(this_df) for this_df in dial]

/home/labicquette/inria/./meetup/code/slurk_utils.py:71: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  this_df['time'] = pd.to_datetime(this_df['time']) - pd.to_datetime(this_df.iloc[0]['time'])


In [49]:
d1 = dial[0]
d2 = dial[1]
d1

,msg,time,user,A_type,B_type,m-type,receiver,A_inst,B_inst,userID
0,,0 days 00:00:00,A,,,join,All,unspec,unspec,112
1,,0 days 00:00:00,B,,,join,All,unspec,unspec,111
2,,0 days 00:00:00,GM,,,join,All,unspec,unspec,97
3,cmd: listen_to,0 days 00:00:01,GM,,,command,All,unspec,unspec,97
4,cmd: listen_to,0 days 00:00:01,GM,,,command,All,unspec,unspec,97
5,cmd: listen_to,0 days 00:00:01,GM,,,command,All,unspec,unspec,97
6,cmd: listen_to,0 days 00:00:01,GM,,,command,All,unspec,unspec,97
7,cmd: listen_to,0 days 00:00:01,GM,,,command,All,unspec,unspec,97
8,cmd: listen_to,0 days 00:00:02,GM,,,command,All,unspec,unspec,97
9,cmd: listen_to,0 days 00:00:02,GM,,,command,All,unspec,unspec,97


In [21]:
image_df = pd.read_csv("LLM-Grounding-Study/data/image_descriptions_all_final.csv")

image_descriptions_dict = {}
for i in range(len(image_df)):
    image_descriptions_dict[image_df.iloc[i]['image_path']] = image_df.iloc[i]['description']

In [22]:
def clean_string(text):
    text = text.replace("/n", "")
    text = text.replace(".w", "")
    text = text.replace(".e", "")
    text = text.replace(".n", "")
    text = text.replace(".s", "")
    text = text.replace("w/", "")
    text = text.replace("e/", "")
    text = text.replace("n/", "")
    text = text.replace("s/", "")
    text = text.replace("[/n]", "")
    text = text.replace("  ", " ")
    text = text.replace(":)", "")
    text = text.replace(";)", "")
    text = text.replace(";-)", "")
    text = text.replace(":D", "")
    text = text.replace(":(", "")
    text = text.replace("no_reply", "")
    text = text.replace("NO_REPLY", "")
    return text

In [23]:
import re
def make_bot_dial(text):
    #text = re.sub('hi','other text',text)
    text = re.sub('^[hH]i(\sthere)*\s*(howdy)*\s*[.,:;!-]*','',text)
    text = re.sub('^[hH]ey(\sthere)*\s*(howdy)*\s*[.,:;!-]*','',text)
    text = re.sub(r'[hH]ello(\sthere)*\s*(howdy)*\s*[.,:;!-]*','',text)
    text = re.sub(r'howdy(\s)*[.,:;!]*','',text)
    return text

def make_top_dial(text):
    #text = re.sub('hi','other text',text)
    return text

In [44]:
def get_raw_dialogue(df):
    res = []
    for i,d in enumerate(df):
        d = d.loc[((d["user"] == "GM") & (d["m-type"] == "new_image")) | ((d["user"] != "GM") & (d["m-type"] == "text"))]
        d.loc[d["msg"].str.contains("url"), "msg"] = [image_descriptions_dict[url.replace(" ", "").split("/")[-1]] for url in d.loc[d["msg"].str.contains("url"), "msg"]]
        
        #d.loc[d["user"] != "GM", "msg"] = d.loc[d["user"] != "GM", "msg"].apply(lambda x: clean_string(x))
        indices = d[d["user"] != "GM"].index[:]
        d.loc[indices, "msg"] = d.loc[indices, "msg"].apply(clean_string)
        #indices = d[d["user"] != "GM"].index[:5]
        # Apply the function to these indices
        #d.loc[indices, "msg"] = d.loc[indices, "msg"].apply(make_top_dial)
        d = d[d["msg"] != ""]
        res += [d]
    return res
#dialogue = get_raw_dialogue(dial)
for i,d in enumerate(dialogue): 
    print("NUMBER DIALOGUE : ", i)
    print(d.loc[d["m-type"] == 'text', "msg"])
    #print(dial[i].loc[(dial[i]["m-type"] == 'text') & (d["user"] != "GM"), "msg"])
    print("\n\n\n\n")

NUMBER DIALOGUE :  0
2                                 i am currently out side
3                                         Im in a kitchen
5          i am now in a dinning room with 4 brown chairs
6           I see a dining room, but definetly not in one
7                                               I'll move
8                                     well 6 chairs sorry
11                 and a plant in the center of the table
13                                  Im moving to fund you
15                                                     ok
17                                        Broken chairs? 
18                                          Glass table? 
19                                no the legs look silver
20                                        glass table yes
21                               with silver bottom to it
22    Brown chairs, silver legs, plant in center of table
23                                                    yes
24                                               Im

In [25]:
pd.concat([dialogue[0], dialogue[1]], ignore_index=True)

,msg,time,user,A_type,B_type,m-type,receiver,A_inst,B_inst,userID
0,"The image showcases a large, modern kitchen with dark wood cabinets and sleek black countertops. The kitchen is well-equipped with a stove top oven positioned under a ventilation fan, a microwave situated above the oven, and a refrigerator placed on the right side of the room.\nThere are several items placed on the countertops, including a bowl, a few apples, and an orange. The kitchen also features a dining table with chairs placed around it. A potted plant adds a touch of greenery to the room, located near the dining table. The overall design and layout of the kitchen create an inviting and functional space for cooking and dining.",0 days 00:00:02,GM,/k/kitchen,,new_image,A,/k/kitchen/ADE_train_00010469.jpg,unspec,97
1,"The image depicts a city street with several cars parked along the sidewalk. The cars are of various sizes and are parked neatly in designated parking spaces. There are at least five cars visible, with one car being particularly large and occupying a significant portion of the sidewalk.\nIn addition to the cars, there are two people in the scene. One person is standing near the middle of the image, while the other is located closer to the right side. A motorcycle is also parked on the street, adding to the variety of vehicles present.\nThe street is lined with trees, providing a pleasant urban atmosphere. The overall scene captures a typical day in a city with parked cars and pedestrians going about their daily activities.",0 days 00:00:03,GM,/k/kitchen,/s/street,new_image,B,/k/kitchen/ADE_train_00010469.jpg,/s/street/ADE_train_00017552.jpg,97
2,i am currently out side,0 days 00:00:19,B,/k/kitchen,/s/street,text,All,/k/kitchen/ADE_train_00010469.jpg,/s/street/ADE_train_00017552.jpg,111
3,Im in a kitchen,0 days 00:00:25,A,/k/kitchen,/s/street,text,All,/k/kitchen/ADE_train_00010469.jpg,/s/street/ADE_train_00017552.jpg,112
4,"The image features a modern dining room with a glass table surrounded by several chairs. A white dog is sitting underneath the table, looking up at the camera. The dining area is adorned with a potted plant placed on the table, adding a touch of greenery to the room.\nThere are wine glasses and a bottle on the table, suggesting that the room is set up for a meal or gathering. The chairs are arranged around the table, with some closer to the foreground and others further in the background. The overall atmosphere of the room is inviting and comfortable.",0 days 00:00:29,GM,/k/kitchen,/d/dining_room,new_image,B,/k/kitchen/ADE_train_00010469.jpg,/d/dining_room/ADE_train_00006891.jpg,97
5,i am now in a dinning room with 4 brown chairs,0 days 00:00:43,B,/k/kitchen,/d/dining_room,text,All,/k/kitchen/ADE_train_00010469.jpg,/d/dining_room/ADE_train_00006891.jpg,111
6,"I see a dining room, but definetly not in one",0 days 00:00:49,A,/k/kitchen,/d/dining_room,text,All,/k/kitchen/ADE_train_00010469.jpg,/d/dining_room/ADE_train_00006891.jpg,112
7,I'll move,0 days 00:00:52,A,/k/kitchen,/d/dining_room,text,All,/k/kitchen/ADE_train_00010469.jpg,/d/dining_room/ADE_train_00006891.jpg,112
8,well 6 chairs sorry,0 days 00:01:03,B,/k/kitchen,/d/dining_room,text,All,/k/kitchen/ADE_train_00010469.jpg,/d/dining_room/ADE_train_00006891.jpg,111
9,"The image features a formal dining room with a black table and six black chairs placed around it. The table is adorned with white flowers in a vase, adding a touch of elegance to the room. The chairs are arranged around the table, with two chairs on each side and one chair at each end.\nA large window can be seen in the dining room, providing natural light and a view of the outside. The room is well-decorated, and the combination of the black furniture and white flowers creates a sophisticated atmosphere.",0 days 00:01:18,GM,/d/dining_room,/d/dining_room,new_image,A,/d/dining_room/ADE_train_00006984.jpg,/d/dining_room/ADE_train_00006891.jpg,97


In [26]:
#[d.to_csv("./data/test/dialogue_"+str(i)+".csv", index=False) for i,d in enumerate(dialogue)]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [3]:
from glob import glob
import re
files = glob("./data/top/*.csv")
files.sort()
print(files)

['./data/top/dialogue_0.csv', './data/top/dialogue_1.csv', './data/top/dialogue_10.csv', './data/top/dialogue_100.csv', './data/top/dialogue_101.csv', './data/top/dialogue_102.csv', './data/top/dialogue_103.csv', './data/top/dialogue_104.csv', './data/top/dialogue_105.csv', './data/top/dialogue_106.csv', './data/top/dialogue_107.csv', './data/top/dialogue_108.csv', './data/top/dialogue_109.csv', './data/top/dialogue_11.csv', './data/top/dialogue_110.csv', './data/top/dialogue_111.csv', './data/top/dialogue_112.csv', './data/top/dialogue_113.csv', './data/top/dialogue_114.csv', './data/top/dialogue_115.csv', './data/top/dialogue_116.csv', './data/top/dialogue_117.csv', './data/top/dialogue_118.csv', './data/top/dialogue_119.csv', './data/top/dialogue_12.csv', './data/top/dialogue_120.csv', './data/top/dialogue_121.csv', './data/top/dialogue_122.csv', './data/top/dialogue_123.csv', './data/top/dialogue_124.csv', './data/top/dialogue_125.csv', './data/top/dialogue_126.csv', './data/top/di

In [163]:
nbfiles = sorted([int(f.split('_')[1].split('.')[0]) for f in files])
dialogue = [pd.read_csv("./data/bot/dialogue_"+ str(i) +".csv") for i in nbfiles]

In [54]:
pd.read_csv("./meetup_grounding/Meetup Dataset/0.csv", index_col="Unnamed: 0")

,time,public,user,receiver,DU ID,Grounding Act,Opened CGUs,Grounded CGUs,Degree
0,0 days 00:00:00,A:,A,All,NaN,NaN,NaN,NaN,NaN
1,0 days 00:00:01,B:,B,All,NaN,NaN,NaN,NaN,NaN
2,0 days 00:00:01,GM:,GM,All,NaN,NaN,NaN,NaN,NaN
27,0 days 00:00:15,B: i'm in the playroom,B,All,[1],['Initiate'],[1],[],[]
31,0 days 00:00:17,B: you have to go west,B,All,[2],['Initiate'],[2],[],[]
32,0 days 00:00:19,A: What does it look like,A,All,"[1, 3]","['Use', 'Initiate']",[3],[1],['medium']
45,0 days 00:00:38,A: describe it please,A,All,"[2, 3]","['Move', 'Repeat']",[],[2],['low']
46,0 days 00:00:55,"A: I'm in one as well, I see a small chalkboard in the corner and a small tent to the right",A,All,[4],['Initiate'],[4],[],[]
47,0 days 00:01:39,A: Please describe what you see in your playroom,A,All,[3],['Repeat'],[],[],[]
48,0 days 00:01:42,A:,A,All,NaN,NaN,NaN,NaN,NaN


In [164]:
#grounds = [pd.read_csv(f) for f in glob("./meetup_grounding/Meetup Dataset/*.csv")]
grounds = []
for i,f in enumerate(glob("./meetup_grounding/Meetup Dataset/*.csv")):
    print(i, f)
    try : 
        grounds += [pd.read_csv(f)]
    except Exception as e:
        grounds += [pd.read_csv(f, delimiter=';')]

#retrieve_grounding(dialogue[0], )

0 ./meetup_grounding/Meetup Dataset/77.csv
1 ./meetup_grounding/Meetup Dataset/414.csv
2 ./meetup_grounding/Meetup Dataset/398.csv
3 ./meetup_grounding/Meetup Dataset/270.csv
4 ./meetup_grounding/Meetup Dataset/209.csv
5 ./meetup_grounding/Meetup Dataset/229.csv
6 ./meetup_grounding/Meetup Dataset/123.csv
7 ./meetup_grounding/Meetup Dataset/56.csv
8 ./meetup_grounding/Meetup Dataset/304.csv
9 ./meetup_grounding/Meetup Dataset/405.csv
10 ./meetup_grounding/Meetup Dataset/42.csv
11 ./meetup_grounding/Meetup Dataset/383.csv
12 ./meetup_grounding/Meetup Dataset/8.csv
13 ./meetup_grounding/Meetup Dataset/49.csv
14 ./meetup_grounding/Meetup Dataset/254.csv
15 ./meetup_grounding/Meetup Dataset/5.csv
16 ./meetup_grounding/Meetup Dataset/95.csv
17 ./meetup_grounding/Meetup Dataset/284.csv
18 ./meetup_grounding/Meetup Dataset/184.csv
19 ./meetup_grounding/Meetup Dataset/362.csv
20 ./meetup_grounding/Meetup Dataset/358.csv
21 ./meetup_grounding/Meetup Dataset/181.csv
22 ./meetup_grounding/Meetup 

In [87]:
grounds[0]

,Unnamed: 0,time,public,user,receiver,DU ID,Grounding Act,Opened CGUs,Grounded CGUs,Degree
0,0.0,0 days 00:00:00,A:,A,All,NaN,NaN,NaN,NaN,NaN
1,1.0,0 days 00:00:00,B:,B,All,NaN,NaN,NaN,NaN,NaN
2,2.0,0 days 00:00:01,GM:,GM,All,NaN,NaN,NaN,NaN,NaN
3,20.0,0 days 00:00:10,A: hi,A,All,NaN,NaN,NaN,NaN,NaN
4,27.0,0 days 00:00:15,B: hi,B,All,NaN,NaN,NaN,NaN,NaN
5,32.0,0 days 00:00:20,B: im in a nursery,B,All,[1],['Initiate'],[1],[],[]
6,33.0,0 days 00:00:22,A: i am in a nursery,A,All,[2],['Initiate'],[2],[],[]
7,34.0,0 days 00:00:25,A: one crib,A,All,"[1, 3]","['Move', 'Initiate']",[3],[1],['low']
8,35.0,0 days 00:00:32,A: it is brown with a pink blanket on it,A,All,[3],['Continue'],[],[],[]
9,36.0,0 days 00:00:32,B: baloon on the crib?,B,All,"[2, 3, 4]","['Move', 'Use', 'Initiate']",[4],"[2, 3]","['low', 'medium']"


In [165]:
def retrieve_time(df, dfs):
    df = df[df["user"] != "GM"]
    for i,d in enumerate(dfs) : 
        d = d[d["user"] != "GM"]
        public = d["time"].dropna()
        if len(public) == 0 :
            continue 
        #public = public.to_list()
        match = True
        for k,msg in enumerate(df["time"].to_list()):
            cond = public.str.contains(msg, regex=False)
            if any(cond):
                users = public[cond].loc[d[cond]["user"] == df["user"].iloc[k]]
                if len(users) == 0 :
                    match = False
                    break
                #else :
                #    message_cond = d["public"].dropna().str.contains(df["msg"].iloc[k], regex=False)
                #    if not any(message_cond):
                #        match = False
                #        break
            else : 
                match = False
                break            

        if match:
            return i
    return -1
time_match = [(i,retrieve_time(d, grounds)) for i,d in enumerate(dialogue)]
print(time_match)

[(0, 112), (1, 223), (2, 119), (3, 64), (4, 269), (5, 371), (6, 212), (7, 227), (8, 13), (9, 408), (10, 85), (11, 99), (12, 313), (13, 12), (14, 57), (15, 56), (16, 184), (17, 34), (18, 121), (19, 271), (20, 10), (21, 166), (22, 33), (23, 181), (24, 219), (25, 163), (26, 185), (27, 392), (28, 356), (29, 155), (30, 132), (31, 136), (32, 15), (33, 31), (34, 79), (35, 366), (36, 339), (37, 376), (38, 246), (39, 60), (40, 150), (41, 369), (42, 337), (43, 386), (44, 335), (45, 178), (46, 221), (47, 122), (48, 264), (49, 224), (50, 165), (51, 326), (52, 230), (53, 24), (54, 263), (55, 319), (56, 140), (57, 191), (58, 340), (59, 58), (60, 177), (61, 293), (62, 74), (63, 70), (64, 282), (65, 68), (66, 90), (67, 187), (68, 37), (69, 317), (70, 348), (71, 349), (72, 231), (73, 291), (74, 14), (75, 415), (76, 423), (77, 391), (78, 107), (79, 259), (80, 416), (81, 252), (82, 54), (83, 213), (84, 145), (85, 204), (86, 318), (87, 283), (88, 241), (89, 16), (90, 396), (91, 151), (92, 158), (93, 407),

In [166]:
import numpy as np

total = set(range(430))
matched = []
nomatch = []
for i,j in time_match :
    if j == -1 : 
        nomatch += [i] #dialogue index
    else : 
        matched += [j] #grounds index


#nomatch dialogues -> nomatch
#nomatch grounds -> total - matched 
#doublons grounds -> np unique -> values[counts>2] 
#doublons dialogues - > doublei 

values, counts = np.unique(matched, return_counts=True) #grounds
print("doublons grounds",values[np.where(counts > 1)]) # doublons
print("nomatch dialogue: ", nomatch) # dialogue[i] with no match w

doublej = values[np.where(counts > 1)]
doublei = []

for i,j in time_match :
    if j in doublej:
        doublei += [i]

print("doublons dialogue ", doublei) #dialogue

nomatch = set(nomatch) # dialogue[i] with no match 
matched = set(matched)

print("nomatch grounds", total - matched) # grounds[i] with no match 

tot_grounds = list(values[np.where(counts > 1)]) + list(total-matched)
tot_dialogue = doublei + list(nomatch)
print(tot_grounds)
print(tot_dialogue)

doublons grounds []
nomatch dialogue:  [262, 271, 414]
doublons dialogue  []
nomatch grounds {40, 48, 109}
[40, 48, 109]
[414, 262, 271]


In [167]:
for i,j in zip(tot_dialogue, tot_grounds):
    print("DIALOGUE : ", i)
    print(dialogue[i].loc[dialogue[i]["user"] != "GM"]["msg"])
    print("\n\n\n")
    print("GROUNDS : ", j)
    print(grounds[j].loc[grounds[j]["user"] != "GM"]["public"])
    print("\n\n\n")

DIALOGUE :  414
3                                                    I see a staircase, it is white and ornate...
8     there is a large spray of flowers in the foreground... and a brown table with little houses
9                i see a stair case with a green rug on it the stair case and banister are wooden
10                                                               think we're in different ones...
12                                                                                          agree
14                                                            want to keep exploring or should I?
16                                                                         ooh i think i found it
17                                                                                     it's white
18                                                                              looks like marble
19                                            hmmm my stairs are brown but the bannister is white
40  

In [168]:
#(dialogue, grounds)
add_indexes = [(53,24),
               (72,231),
               (124,25),
               (215,250),
               (224,134),
               (232, 304),
               (257, 106),
               (351, 330),
               (370, 19),
               (383,115),
               (414, 109),
               (262, 40),
               (271, 48),
               ]

add_indexes = [(414, 109),
               (262, 40),
               (271, 48)]

In [171]:
test_i = []
test_j = []
for i,j in time_match:
    #time_match[i] = (i,j)
    test_i += [i]
    test_j += [j]
    
print(len(np.unique(test_i)))
print(len(np.unique(test_j)))

430
430


In [ ]:
def retrieve_time(df, dfs):
    df = df[df["user"] != "GM"]
    for i,d in enumerate(dfs) : 
        d = d[d["user"] != "GM"]
        public = d["time"].dropna()
        if len(public) == 0 :
            continue 
        #public = public.to_list()
        match = True
        for k,msg in enumerate(df["time"].to_list()):
            cond = public.str.contains(msg, regex=False)
            if any(cond):
                users = public[cond].loc[d[cond]["user"] == df["user"].iloc[k]]
                if len(users) == 0 :
                    match = False
                    break
                #else :
                #    message_cond = d["public"].dropna().str.contains(df["msg"].iloc[k], regex=False)
                #    if not any(message_cond):
                #        match = False
                #        break
            else : 
                match = False
                break            

        if match:
            return i
    return -1

In [172]:
i_index =[]
j_index =[]
for i,j in time_match:
    dialogue[i]["DU ID"] = [[] for _ in range(len(dialogue[i]))]
    dialogue[i]["Grounding Act"] = [[] for _ in range(len(dialogue[i]))]
    dialogue[i]["Opened CGUs"] = [[] for _ in range(len(dialogue[i]))]
    dialogue[i]["Grounded CGUs"] = [[] for _ in range(len(dialogue[i]))]
    dialogue[i]["Degree"] = [[] for _ in range(len(dialogue[i]))]
    d = dialogue[i].loc[dialogue[i]["user"] != "GM"]
    g = grounds[j].loc[grounds[j]["user"] != "GM"]
    for k, t1 in enumerate(d['time']):
        for l, t2 in enumerate(g['time']):
            if t1 == t2 : 
                d.loc[d.index[k],"DU ID"] = g.loc[g.index[l],'DU ID']
                d.loc[d.index[k],"Grounding Act"] = g.loc[g.index[l],'Grounding Act']
                d.loc[d.index[k],"Opened CGUs"] = g.loc[g.index[l],'Opened CGUs']
                d.loc[d.index[k],"Grounded CGUs"] = g.loc[g.index[l],'Grounded CGUs']
                d.loc[d.index[k],"Degree"] = g.loc[g.index[l],'Degree']
                
    
    d.to_csv("./data/bot_merge/dialogue_"+ str(i) +".csv", index=False)

In [6]:
import numpy as np
import time
import Levenshtein
def retrieve_grounding(df, dfs):
    df = df[df["user"] != "GM"]
    for i,d in enumerate(dfs) : 
        d = d[d["user"] != "GM"]
        cpt =0
        for j,s in enumerate(d["public"]):
            if type(s) != str:
                continue
            
            if (s == "A: ") or (s == "B: ") or (s == "GM: "):
                    continue
            if ("hi" in s) or ("hello" in s) or ("ok" in s): 
                    continue
            for k,msg in enumerate(df["msg"].to_list()):
                #if msg in s:
                if list(df["user"])[k] != list(d["user"])[j]:
                    continue
                if Levenshtein.ratio(msg,s) > 0.7:
                    #print()
                    if df["time"].to_list()[k] == d["time"].to_list()[j]:  
                        #print(msg, s)
                        #print("match !", i)
                        cpt += 1
                        if cpt > 2:
                            print(i)
                            return i 
    return -1

matching = [(i,retrieve_grounding(d, grounds)) for i,d in enumerate(dialogue)]


112
223
119
64
269
371
212
227
13
408
99
313
12
57
56
184
34
271
10
181
219
163
185
392
356
155
132
136
15
31
79
339
376
246
60
150
369
337
386
335
178
221
122
264
224
165
326
230
24
263
319
140
191
340
58
177
293
74
70
282
68
90
187
37
317
348
349
291
14
415
423
391
107
259
416
252
54
213
145
204
318
283
241
16
396
151
158
407
186
218
95
208
380
399
381
429
103
305
172
76
400
422
41
43
89
377
316
363
383
384
257
53
105
156
275
206
25
325
45
128
262
94
52
143
207
92
403
120
111
299
297
180
279
374
214
32
372
425
17
395
365
130
334
288
420
238
176
341
244
300
294
343
148
360
183
51
168
236
323
77
428
342
29
281
361
385
160
362
296
268
23
331
124
61
424
247
280
427
28
419
338
101
216
298
84
368
322
93
409
421
388
171
194
116
211
46
225
59
38
157
210
233
133
324
195
174
332
125
3
154
18
35
197
379
367
254
251
295
364
234
170
306
239
49
256
266
235
147
82
65
192
217
240
131
63
310
307
106
198
253
248
139
382
290
426
229
110
108
173
378
286
188
8
414
20
351
96
47
249
346
261
50
152
237
390


In [9]:
total = set(range(430))
matched = []
nomatch = []
for i,j in matching :
    if j == -1 : 
        nomatch += [i]
    else : 
        matched += [j]
nomatch = set(nomatch) # dialogue[i] with no match 
matched = set(matched)
print(nomatch)
print(total - matched) # grounds[i] with no match 
print(len(nomatch))
print(len(matched))

{261, 262, 389, 10, 271, 400, 18, 404, 21, 22, 149, 406, 414, 418, 35, 421, 166, 174, 180, 311, 72, 215, 216, 351, 224, 376, 232, 235, 364, 109, 242, 373, 246, 120, 380, 125, 383}
{1, 6, 134, 267, 397, 142, 401, 153, 27, 284, 285, 33, 166, 167, 40, 169, 48, 304, 308, 62, 328, 330, 333, 85, 86, 88, 353, 355, 231, 109, 366, 115, 245, 375, 121, 250, 255}
37
393


In [11]:
dialogue[261].loc[dialogue[261]["user"] != "GM"] 

,msg,time,user,A_type,B_type,m-type,receiver,A_inst,B_inst,userID
2,hi,0 days 00:00:12,A,/b/bedroom,/k/kitchen,text,All,/b/bedroom/ADE_train_00003878.jpg,/k/kitchen/ADE_train_00010175.jpg,905
3,hi there. I am in kitchen so i will move. Where are you,0 days 00:00:23,B,/b/bedroom,/k/kitchen,text,All,/b/bedroom/ADE_train_00003878.jpg,/k/kitchen/ADE_train_00010175.jpg,906
6,I'm in a bedroom. let's just look until we find it,0 days 00:00:45,A,/b/bedroom,/d/doorway/indoor,text,All,/b/bedroom/ADE_train_00003878.jpg,/d/doorway/indoor/ADE_train_00007338.jpg,905
11,Ok I think I might have it. It is a door way with square windows throughout door and a window at each side of the door. Cream walls and brown wood around window. Have a look for that,0 days 00:01:29,B,/d/doorway/indoor,/d/doorway/indoor,text,All,/d/doorway/indoor/ADE_train_00007330.jpg,/d/doorway/indoor/ADE_train_00007338.jpg,906
12,I found a doorway but it doesn't look like that,0 days 00:01:50,A,/d/doorway/indoor,/d/doorway/indoor,text,All,/d/doorway/indoor/ADE_train_00007330.jpg,/d/doorway/indoor/ADE_train_00007338.jpg,905
13,ok do you want to look for mine or do you want me to move to look for yours?,0 days 00:02:06,B,/d/doorway/indoor,/d/doorway/indoor,text,All,/d/doorway/indoor/ADE_train_00007330.jpg,/d/doorway/indoor/ADE_train_00007338.jpg,906
14,I'll keep looking. stay there,0 days 00:02:10,A,/d/doorway/indoor,/d/doorway/indoor,text,All,/d/doorway/indoor/ADE_train_00007330.jpg,/d/doorway/indoor/ADE_train_00007338.jpg,905
17,ok.,0 days 00:02:20,B,/d/doorway/indoor,/d/doorway/indoor,text,All,/d/doorway/indoor/ADE_train_00007330.jpg,/d/doorway/indoor/ADE_train_00007338.jpg,906
19,Are the windows square at the bottom and rounded to a point at the top,0 days 00:03:05,A,/d/doorway/indoor,/d/doorway/indoor,text,All,/d/doorway/indoor/ADE_train_00007338.jpg,/d/doorway/indoor/ADE_train_00007338.jpg,905
20,the door has windows throughout. The side windows are pointed,0 days 00:03:25,B,/d/doorway/indoor,/d/doorway/indoor,text,All,/d/doorway/indoor/ADE_train_00007338.jpg,/d/doorway/indoor/ADE_train_00007338.jpg,906


In [59]:
print(len(matching))
values, counts = np.unique(matching, return_counts=True)
print(print(len(values), counts))

430
391 [40  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1 

In [ ]:
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
def combine(n, t):
    for i in range(t):
        top = i%t
        rand = np.random.choice()
        np.random

In [46]:
#dialogue = get_raw_dialogue(dialogue)

In [47]:
#[d.to_csv("./data/mid/dialogue_"+str(i)+".csv", index=False) for i,d in enumerate(dialogue)]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,